In [ ]:
!git clone https://github.com/alexa/Topical-Chat.git

Cloning into 'Topical-Chat'...
remote: Enumerating objects: 262, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 262 (delta 26), reused 15 (delta 3), pack-reused 197
Receiving objects: 100% (262/262), 96.25 MiB | 10.19 MiB/s, done.
Resolving deltas: 100% (134/134), done.
Updating files: 100% (119/119), done.


In [ ]:
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install sentencepiece


In [ ]:
!pip install transformers[torch]

In [ ]:
import pandas as pd
import json
import torch
from transformers import BertTokenizer, T5Tokenizer, BertForSequenceClassification, T5ForConditionalGeneration, AdamW

from tqdm import tqdm
from transformers import T5Tokenizer, T5ForConditionalGeneration, DataCollatorForSeq2Seq, TrainingArguments, Trainer
import json
from transformers import pipeline
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader, Dataset


In [ ]:
# Load the JSON dataset
dataset_path = "/content/drive/MyDrive/NLPdataset/Topical-Chat/conversations/train.json"

with open(dataset_path, "r") as json_file:
    chat_data = json.load(json_file)


In [ ]:
# Create a T5 tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")


/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:220: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes 

In [ ]:

# Define a mapping from sentiment labels to numeric values
sentiment_label_map = {
    "Angry": 0,
    "Curious to dive deeper": 1,
    "Disgusted": 2,
    "Fearful": 3,
    "Happy": 4,
    "Sad": 5,
    "Surprised": 6,
    "Neutral" : 7
}


In [ ]:
# Define a custom dataset class
class CustomChatDataset(Dataset):
    def __init__(self, data, tokenizer, max_seq_length=128):
        self.data = list(data.values())
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        messages = item['content']
        input_text = ' '.join([f"{msg['message']}" for msg in messages])
        target_text = messages[-1]['message']
        # # Add a placeholder for sentiment prediction
        # input_text = f"User: {input_text} [SENTIMENT]"

        # input_tokens = self.tokenizer.encode(input_text, add_special_tokens=True, max_length=self.max_seq_length, truncation=True, padding='max_length', return_tensors='pt')

        # return {
        #     "input_ids": input_tokens[0],
        #     "decoder_input_ids" : input
        # }

        # Tokenize input text
        input_tokens = self.tokenizer.encode(input_text,
                                             add_special_tokens=True,
                                             max_length=self.max_seq_length,
                                             truncation=True,
                                             padding='max_length',
                                             return_tensors='pt')

        # Tokenize target text (agent_2's message)
        target_tokens = self.tokenizer.encode(target_text,
                                              add_special_tokens=False,
                                              max_length=self.max_seq_length,
                                              truncation=True,
                                              padding='max_length',
                                              return_tensors='pt')

        return {
            "input_ids": input_tokens[0],  # Remove the batch dimension
            "decoder_input_ids": target_tokens[0],  # Remove the batch dimension
        }

# Create a CustomChatDataset
chat_dataset = CustomChatDataset(chat_data, tokenizer)

In [ ]:

# Create a DataLoader
batch_size = 4
chat_dataloader = DataLoader(chat_dataset, batch_size=batch_size, shuffle=True)



In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/NLP/ChatbotModel",
    per_device_train_batch_size=batch_size,
    num_train_epochs=3,
    save_steps=1000,
    save_total_limit=2,
    remove_unused_columns=False,
    report_to="tensorboard",
)

In [ ]:
# Create a T5 model for sequence-to-sequence
chatbot_model = T5ForConditionalGeneration.from_pretrained("t5-base")

# Create a data collator for sequence-to-sequence tasks
custom_data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=chatbot_model,
    padding=True,
    max_length=128,
)

# Create a Trainer for the chatbot model
chatbot_trainer = Trainer(
    model=chatbot_model,
    args=training_args,
    data_collator=custom_data_collator,
    tokenizer=tokenizer,
    train_dataset=chat_dataset,
)

In [ ]:
# Define the loss function for custom training
def custom_loss(model, inputs):
    outputs = model(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        decoder_input_ids=inputs["decoder_input_ids"],
        labels=inputs["decoder_input_ids"],
    )
    loss = outputs.loss
    return loss

chatbot_trainer.compute_loss = custom_loss

# Start training the chatbot model
chatbot_trainer.train()

# Save the trained chatbot model
chatbot_trainer.save_model()


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2436: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Step,Training Loss
500,0.591900


KeyboardInterrupt: ignored

In [ ]:
# Function for generating responses
def generate_response(input_text):
    # Detect sentiment using VADER sentiment analysis
    sentiment_scores = analyzer.polarity_scores(input_text)

    # Choose an appropriate sentiment label based on VADER scores
    compound_score = sentiment_scores['compound']
    if compound_score >= 0.05:
        sentiment_label = "positive"
    elif compound_score <= -0.05:
        sentiment_label = "Curious to dive deeper"
    else:
        sentiment_label = "neutral"

    # Map the sentiment label to the corresponding label used in your dataset
    sentiment_label_mapped = sentiment_mapping.get(sentiment_label, "Neutral")  # Default to "Neutral" if not found

    # Generate a response based on the detected sentiment
    user_input = f"User: {input_text}"
    sentiment_text = f"[SENTIMENT: {sentiment_label_mapped}]"
    input_ids = tokenizer.encode(user_input, sentiment_text,
                                 return_tensors="pt", max_length=128,
                                 truncation=True, padding=True,
                                 tgt_lang="en",  # Set target language to English
                                )

    response_ids = model.generate(
        input_ids=input_ids,
        max_length=128,
        num_return_sequences=1,
        no_repeat_ngram_size=2,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
        tgt_lang="en",
    )

In [ ]:
def generate_chat_response(input_text):
    input_text = f"User: {input_text} [SENTIMENT]"

    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=128, truncation=True, padding=True)

    response_ids = chatbot_model.generate(input_ids, max_length=128, num_return_sequences=1, decoder_input_ids=input_ids)  # Specify decoder_input_ids
    chatbot_response = tokenizer.decode(response_ids[0], skip_special_tokens=True)

    # Extract the predicted sentiment from the response
    predicted_sentiment = chatbot_response.split("[SENTIMENT]")[1].strip()
    return chatbot_response, predicted_sentiment

chatbot_model.to("cpu")

# Example of generating a chatbot response
user_input = "Do you like dance?"
chat_response, predicted_sentiment = generate_chat_response(user_input)

# Print the chatbot's response and predicted sentiment
print("Chatbot Response:", chat_response)
print("Predicted Sentiment:", predicted_sentiment)

In [ ]:
# Initialize a sentiment analysis pipeline
nlp = pipeline("sentiment-analysis")

# Create lists to store manual and automated sentiment labels
manual_labels = []
automated_labels = []

# Extract sentiments and messages from the dataset
for conversation in dataset:
    for message in conversation['content']:
        user_message = message["message"]
        sentiment = message["sentiment"]

        # Perform automated sentiment analysis using the pipeline
        automated_sentiment = nlp(user_message)[0]['label']

        # Append to the lists
        manual_labels.append(sentiment)
        automated_labels.append(automated_sentiment)

# Create a mapping from manual labels to class indices (if needed)
label_to_index = {label: i for i, label in enumerate(set(manual_labels))}

# Convert labels to indices
manual_labels = [label_to_index[label] for label in manual_labels]

# Evaluate the performance using classification report
report = classification_report(manual_labels, automated_labels, target_names=list(set(manual_labels)), output_dict=True)

# Print the classification report
print("Classification Report:")
print(json.dumps(report, indent=4))